In [437]:
import json
import math

import numpy as np
import pandas as pd

 0. для работы с нечеткими числами создадим некоторый вспомогательный класс

In [438]:
class FuzzyNumber:
    def __init__(self, a, b, c):
        super(FuzzyNumber, self).__init__()
        self.c = c
        self.b = b
        self.a = a

    def str(self):
        return f"<{self.a}, {self.b}, {self.c}>"

    def __repr__(self):
        return self.str()
        # return f"FuzzyNumber(a={self.a}, b={self.b}, c={self.c})"

    def to_json(self):
        return {'a': self.a, 'b': self.b, 'c': self.c}

    @classmethod
    def from_json(cls, json_data):
        return cls(**json_data)

    def __ge__(self, other):
        return self.c > other.c

    def __lt__(self, other):
        return self.c < other.c

    def __truediv__(self, other):
        if isinstance(other, float):
            return FuzzyNumber(self.a / other, self.b / other, self.c / other)
        return FuzzyNumber(self.a / other.a, self.b / other.b, self.c / other.c)

    def __mul__(self, other):
        return FuzzyNumber(self.a * other.a, self.b * other.b, self.c * other.c)

    def __matmul__(self, other):
        temp = math.hypot(self.a, other.a) + math.hypot(self.b, other.b) + math.hypot(self.c, other.c)
        return math.sqrt(temp / 3)

    def __add__(self, other):
        return FuzzyNumber(self.a + other.a, self.b + other.b, self.c + other.c)

    def __floordiv__(self, other):
        if isinstance(other, float):
            return FuzzyNumber(other / self.c, other / self.b, other / self.a)


## 1. Подготовка данных:
 Необходимо собрать оценки от всех членов группы для каждого критерия и каждой альтернативы. Оценки должны быть представлены в форме нечетких чисел. Здесь, например, используются треугольные

In [439]:
def parse():
    with open('data.json', 'r+') as file:
        data = json.load(file)

    for expert_ratings in data:
        for k, alternatives in expert_ratings.items():
            for k1, c in alternatives.items():
                expert_ratings[k][k1] = FuzzyNumber.from_json(c)
    return data

In [440]:
alternatives = ["Waterfall", "Agile", 'PMI']
criterias = ["flexibility", "quality", "time", "cost"]

## 2. Агрегация оценок
Следующим шагом необходимо агрегировать индивидуальные оценки членов группы для формирования коллективной оценочной матрицы

In [441]:
# Функция для агрегации оценок
def aggregate_ratings(ratings):
    # Создаем пустой DataFrame с индексами соответствующими критериям
    df = pd.DataFrame(index=alternatives, columns=criterias)

    for alternative in alternatives:
        for criteria in criterias:
            experts = len(ratings)
            # Используем np.array вместо np.zeros
            r_ = np.array([[0, 0, 0]] * experts, dtype=float)
            for expert in range(experts):
                fn = ratings[expert][alternative][criteria]
                r_[expert, 0] = fn.a
                r_[expert, 1] = fn.b
                r_[expert, 2] = fn.c
            # Используем min, mean и max вместо argmin, mean и argmax
            cell = FuzzyNumber(np.min(r_[:, 0]), np.mean(r_[:, 1]), np.max(r_[:, 2]))
            df.at[alternative, criteria] = cell  # Используйте at для обращения к элементам DataFrame

    return df

In [442]:
df = aggregate_ratings(parse())
df

,flexibility,quality,time,cost
Waterfall,"<1.0, 3.0, 7.0>","<3.0, 5.5, 8.0>","<3.0, 5.0, 7.0>","<3.0, 5.5, 8.0>"
Agile,"<6.0, 7.5, 9.0>","<6.0, 7.5, 9.0>","<1.0, 2.5, 8.0>","<1.0, 2.5, 8.0>"
PMI,"<3.0, 6.5, 9.0>","<6.0, 7.75, 9.0>","<1.0, 1.0, 3.0>","<1.0, 4.0, 7.0>"


## 3.Нормализация агрегированной оценочной матрицы.

In [443]:
temp_1 = [fn.max().c for fn in df[["flexibility", "quality"]].to_numpy().T]  # benefit criterias
temp_2 = [fn.min().c for fn in df[["time", "cost"]].to_numpy().T]  # cost criterias
norm_coeffs = np.array(temp_1 + temp_2)
norm_coeffs

array([9., 9., 3., 7.])

In [444]:
p1 = df[["flexibility", "quality"]] / temp_1
p2 = df[["time", "cost"]] // temp_2
normalized_df = pd.merge(p1, p2, left_index=True, right_index=True)
normalized_df

,flexibility,quality,time,cost
Waterfall,"<0.1111111111111111, 0.3333333333333333, 0.777...","<0.3333333333333333, 0.6111111111111112, 0.888...","<0.42857142857142855, 0.6, 1.0>","<0.875, 1.2727272727272727, 2.3333333333333335>"
Agile,"<0.6666666666666666, 0.8333333333333334, 1.0>","<0.6666666666666666, 0.8333333333333334, 1.0>","<0.375, 1.2, 3.0>","<0.875, 2.8, 7.0>"
PMI,"<0.3333333333333333, 0.7222222222222222, 1.0>","<0.6666666666666666, 0.8611111111111112, 1.0>","<1.0, 3.0, 3.0>","<1.0, 1.75, 7.0>"


In [445]:
normalized_matrix = normalized_df.to_numpy()

## 4. Вычисление взвешенной нормализованной матрицы путем умножения на веса критериев

In [446]:
weights = [FuzzyNumber(5, 7, 8), FuzzyNumber(7, 7, 8), FuzzyNumber(6, 7, 9), FuzzyNumber(5, 6, 7)]
weighted_matrix = normalized_matrix * weights
df = pd.DataFrame(weighted_matrix, index=alternatives, columns=criterias)
df

,flexibility,quality,time,cost
Waterfall,"<0.5555555555555556, 2.333333333333333, 6.2222...","<2.333333333333333, 4.277777777777779, 7.11111...","<2.571428571428571, 4.2, 9.0>","<4.375, 7.636363636363637, 16.333333333333336>"
Agile,"<3.333333333333333, 5.833333333333334, 8.0>","<4.666666666666666, 5.833333333333334, 8.0>","<2.25, 8.4, 27.0>","<4.375, 16.799999999999997, 49.0>"
PMI,"<1.6666666666666665, 5.055555555555555, 8.0>","<4.666666666666666, 6.027777777777779, 8.0>","<6.0, 21.0, 27.0>","<5.0, 10.5, 49.0>"


## 5. Вычисление идеальных и анти-идеальных решений

In [447]:
A_star = [fn.max() for fn in df.to_numpy().T]
A_minus = [fn.min() for fn in df.to_numpy().T]
a_df = pd.DataFrame([A_star, A_minus], index=['A*', 'A-'], columns=criterias)
pd.concat([df, a_df], ignore_index=False)

,flexibility,quality,time,cost
Waterfall,"<0.5555555555555556, 2.333333333333333, 6.2222...","<2.333333333333333, 4.277777777777779, 7.11111...","<2.571428571428571, 4.2, 9.0>","<4.375, 7.636363636363637, 16.333333333333336>"
Agile,"<3.333333333333333, 5.833333333333334, 8.0>","<4.666666666666666, 5.833333333333334, 8.0>","<2.25, 8.4, 27.0>","<4.375, 16.799999999999997, 49.0>"
PMI,"<1.6666666666666665, 5.055555555555555, 8.0>","<4.666666666666666, 6.027777777777779, 8.0>","<6.0, 21.0, 27.0>","<5.0, 10.5, 49.0>"
A*,"<1.6666666666666665, 5.055555555555555, 8.0>","<4.666666666666666, 6.027777777777779, 8.0>","<6.0, 21.0, 27.0>","<5.0, 10.5, 49.0>"
A-,"<0.5555555555555556, 2.333333333333333, 6.2222...","<2.333333333333333, 4.277777777777779, 7.11111...","<2.571428571428571, 4.2, 9.0>","<4.375, 7.636363636363637, 16.333333333333336>"


## 6. Расчета расстояний и определение относительной близости к идеальному решению для каждой альтернативы.

In [448]:
d_star = [sum([weighted_matrix[i][j] @ vj_star for j, vj_star in enumerate(A_star)]) for i in range(len(alternatives))]
d_minus = [sum([weighted_matrix[i][j] @ vj_minus for j, vj_minus in enumerate(A_minus)]) for i in range(len(alternatives))]

In [449]:
d_star = np.array(d_star)
d_minus = np.array(d_minus)

In [450]:
CC = d_minus / (d_minus + d_star)
ranks = np.argsort(CC)[::-1] + 1

In [451]:
final_df = pd.DataFrame(np.array([d_star, d_minus, CC, ranks]).T, index=alternatives, columns=['d*', 'd-', 'CC', 'Rank'])
final_df

,d*,d-,CC,Rank
Waterfall,14.410475,10.997861,0.432845,3.0
Agile,16.098047,14.098849,0.466897,2.0
PMI,16.162526,14.410475,0.471346,1.0
